In [ ]:
!pip install nlpaug
!pip install sacremoses
!pip install gensim


In [ ]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import math
from tqdm.notebook import tqdm
import torch
import gc

In [ ]:
# Back translation using language models from University of Helsinki (https://huggingface.co/Helsinki-NLP)
def back_translation(dataframe, lang_list, num):
    lang_list = lang_list[:num]
    if num<dataframe.shape[0]:
      dataframe = dataframe[:num] #slice num samples from dataframe

    data = dataframe['Clinical history'].tolist()
    new_augmented_data = []
    new_dataframe = dataframe.iloc[:0]
    num_loops = min(math.ceil(num/len(data)),20) # Iterations capped at 20 due to GPU constraints

    # Back translation, using 1 language per loop across available dataset
    # e.g. if only 10 original samples, and producing 20 backtranslations = 2 languages
    for i in range(num_loops):
        from_model = 'Helsinki-NLP/opus-mt-en-' + lang_list[i]
        to_model = 'Helsinki-NLP/opus-mt-' + lang_list[i] + '-en'
        aug = naw.BackTranslationAug(from_model_name=from_model, to_model_name=to_model,
                                    name='BackTranslationAug', device='cuda', batch_size=32)
        augmented_data = aug.augment(data)
        augmented_df = pd.DataFrame(augmented_data,columns=['Clinical history'])

        # Original datapoints are copied to retain all the non-free text parameters e.g. Age
        dataframe_copy = dataframe.copy()

        # Replace the clinical history with backtranslated text and added to new dataframe
        dataframe_copy.loc[:,'Clinical history'] = augmented_df['Clinical history'].values
        new_dataframe = pd.concat([new_dataframe, dataframe_copy], axis=0)

        if i%10 == 0:
            gc.collect()
            torch.cuda.empty_cache()

    return new_dataframe


In [ ]:
'''
DATA AUGMENTATION

Please replace file_path with the path containing the training data.
set_sample = True if using sample data only.
'''

file_path = '/content/drive/MyDrive/Colab Notebooks/Final Notebooks/Submission/'
set_sample = True

# Import training data for augmentation
df = pd.read_csv(file_path + "TrainData70-30.csv")

# List of languages for back translation from University of Helsinki (https://huggingface.co/Helsinki-NLP)
# nlp_languages.csv contains a list of codes for languages

nlpaug_languages = pd.read_csv(file_path + "Data/nlp_languages.csv", header=None)
language_list = nlpaug_languages[0].tolist()

label_type = 'Protocol'

frequency_table = df[label_type].value_counts()

# print(frequency_table.to_string())


In [ ]:
'''
CREATE DATAFRAMES FOR AUGMENTATION

Augments data by num_aug_label samples, below a certain number of samples (freq_threshold).
See manuscript Methods for details.
Divides the number of required augmented samples into the number of augmentation techniques available.

Note: Backtranslation limited to 20 languages per protocol due to Colab GPU memory constraints.
e.g. if number of samples for a label is 5, the number of backtranslations
for that label is capped at 20 per sample (100). The remainder of augmentations is split among the
other augmentation techniques.

'''
if set_sample:
    freq_parameters = {'Urg_threshold': 5, 'Urg_num_aug_label': 5, 'Protocol_threshold':5, 'Protocol_num_aug_label': 5}
else:
    freq_parameters = {'Urg_threshold': 2000, 'Urg_num_aug_label': 2000, 'Protocol_threshold':1000, 'Protocol_num_aug_label': 400}

freq_threshold = freq_parameters[str(label_type) + '_threshold']
num_aug_label = freq_parameters[str(label_type) + '_num_aug_label']

num_aug_techniques = 5

# Create dataframe with samples below the specific freq_threshold
new_df = df.iloc[:0]
infreq_df = df[label_type].value_counts().loc[lambda x: x<freq_threshold].reset_index()
infreq_df['backtranslation'] = num_aug_label/num_aug_techniques

# Creates column for number of backtranslation augmentations for each label, capped at 20 different languages
infreq_df['backtranslation'] = np.where(infreq_df['backtranslation']/infreq_df['count']<20,
                                        num_aug_label/num_aug_techniques , 20).astype(int)

# Creates column for number of other non-backtranslation augmentations for each label
infreq_df['other techniques'] = ((num_aug_label - infreq_df['backtranslation']) / 4).astype(int)
infreq_list = infreq_df.values.tolist()
# print(infreq_list)

# Create empty dataframes for each type of augmentation
df_backtranslation = df.iloc[:0]
df_context_word = df.iloc[:0]
df_word_embed = df.iloc[:0]
df_context_sentence = df.iloc[:0]
df_random = df.iloc[:0]

In [ ]:
'''
Iterates through each label type:
1. Slices number of backtranslation samples and performs backtranslation. Adds to df_backtranslation
2. Divides remainder of samples into dataframes for other augmentation technique and adds to respective dataframe
'''
for p in tqdm(infreq_list): #for each label with n samples

    df_protocol = df[df[label_type] == p[0]].copy()
    df_protocol = df_protocol.sample(frac=1) # shuffle
    if p[1] >= num_aug_label:
      # slice required number of samples to augment
      df_aug_label = df_protocol[:num_aug_label]
    else:
      # concatenate label groups and slice required number samples
      num_of_df = math.ceil(num_aug_label/p[1])
      df_protocol_concat = df_protocol.iloc[:0]
      for i in range(num_of_df):
        df_protocol_concat = pd.concat([df_protocol_concat, df_protocol])
      df_aug_label = df_protocol_concat[:num_aug_label]

    # Backtranslation
    augmented_bt = back_translation(df_protocol, language_list, p[2])
    df_backtranslation = pd.concat([df_backtranslation, augmented_bt])

    # Create dataframes for other augmentation techniques
    df_context_word = pd.concat([df_context_word, df_aug_label[p[2]:p[3]*2]])
    df_word_embed = pd.concat([df_word_embed, df_aug_label[p[3]*2:p[3]*3]])
    df_context_sentence = pd.concat([df_context_sentence, df_aug_label[p[3]*3:p[3]*4]])
    df_random = pd.concat([df_random, df_aug_label[p[3]*4:]])


df_backtranslation.to_csv(file_path + "AugmentedBackTranslation.csv", encoding='utf-8', index=False)
df_context_word.to_csv(file_path + "ContextWord.csv", encoding='utf-8', index=False)
df_random.to_csv(file_path + "Random.csv", encoding='utf-8', index=False)
df_context_sentence.to_csv(file_path + "ContextSentence.csv", encoding='utf-8', index=False)
df_word_embed.to_csv(file_path + "WordEmbed.csv", encoding='utf-8', index=False)


In [ ]:
# Exits
if not set_sample:
  import os
  os._exit(00)

**SECTION 2: STOP AND RESTART NOTEBOOK**


In [ ]:
'''
Due to GPU memory constraints on Google colab, after backtranslation data is
saved as .csv and notebook restarted from below for subsequent augmentation techniques
'''

file_path = '/content/drive/MyDrive/Colab Notebooks/Final Notebooks/Submission/'
set_sample = True

from nlpaug.util.file.download import DownloadUtil
DownloadUtil.download_fasttext('wiki-news-300d-1M', dest_dir=file_path)

import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import math
from tqdm.notebook import tqdm
import torch
import gc
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# Word level augmentation using contextual word embedding (bert model)

def context_word_embedding(dataframe, model = 'bert-base-uncased'):

    data = dataframe['Clinical history'].tolist()

    aug = naw.ContextualWordEmbsAug(
        model_path=model, action='substitute',
        name='ContextualWordEmbs_Aug', stopwords='[UNK]', device='cuda', batch_size=32)

    augmented_data = aug.augment(data)
    augmented_df = pd.DataFrame(augmented_data,columns=['Clinical history'])

    # Replace the clinical history with augmented text and added to new dataframe
    dataframe.loc[:,'Clinical history'] = augmented_df['Clinical history'].values

    gc.collect()
    torch.cuda.empty_cache()

    return dataframe

In [ ]:
# Sentence level augmentation using contextual word embedding (gpt2 model)

def context_word_embed_sentence(dataframe):

    data = dataframe['Clinical history'].tolist()
    augmented_data = []

    aug = nas.ContextualWordEmbsForSentenceAug(
        model_path='gpt2', device='cuda')

    for text in tqdm(data):
      augmented_text = aug.augment(text)
      augmented_data.append(augmented_text)

    augmented_df = pd.DataFrame(augmented_data,columns=['Clinical history'])

    # Replace the clinical history with augmented text and added to new dataframe
    dataframe.loc[:,'Clinical history'] = augmented_df['Clinical history'].values

    gc.collect()
    torch.cuda.empty_cache()

    return dataframe

In [ ]:
# Word substitution augmentation using fasttext word embeddings

def word_embedding(dataframe):

    data = dataframe['Clinical history'].tolist()
    augmented_data = []

    aug = naw.WordEmbsAug(
        model_type='fasttext', model_path= file_path +'wiki-news-300d-1M.vec',
        top_k=20, action="substitute")

    for text in tqdm(data):
      augmented_text = aug.augment(text)
      augmented = augmented_data.append(augmented_text)

    augmented_df = pd.DataFrame(augmented_data,columns=['Clinical history'])

    # Replace the clinical history with backtranslated text and added to new dataframe
    dataframe.loc[:,'Clinical history'] = augmented_df['Clinical history'].values

    return dataframe


In [ ]:
# Random word substitution

def random_word(dataframe):

    data = dataframe['Clinical history'].tolist()
    augmented_data = []

    aug = naw.RandomWordAug(action='swap')

    for text in tqdm(data):
      augmented_text = aug.augment(text)
      augmented = augmented_data.append(augmented_text)

    augmented_df = pd.DataFrame(augmented_data,columns=['Clinical history'])

    # Replace the clinical history with backtranslated text and added to new dataframe
    dataframe.loc[:,'Clinical history'] = augmented_df['Clinical history'].values

    return dataframe

In [ ]:

# Continue to process with the remaining data augmentation techniques
df_backtranslation = pd.read_csv(file_path + "AugmentedBackTranslation.csv")
df_context_word = pd.read_csv(file_path + "ContextWord.csv")
df_random = pd.read_csv(file_path + "Random.csv")
df_context_sentence = pd.read_csv(file_path + "ContextSentence.csv")
df_word_embed = pd.read_csv(file_path + "WordEmbed.csv")


df_context_word = context_word_embedding(df_context_word, 'bert-base-uncased')
df_word_embed = word_embedding(df_word_embed)
df_context_sentence = context_word_embed_sentence(df_context_sentence)
df_random = random_word(df_random)

In [ ]:
'''
COMBINE AUGMENTED DATA WITH ORIGINAL TRAINING DATA
'''
# Import training data for augmentation
df = pd.read_csv(file_path + "TrainData70-30.csv")

augmented_df = pd.concat([df_backtranslation, df_context_word, df_context_sentence, df_word_embed, df_random])

augmented_df.to_csv(file_path + "AugmentedData.csv", encoding='utf-8', index=False)

combined_aug_train_df = pd.concat([df, augmented_df])
combined_aug_train_df = combined_aug_train_df.sample(frac=1)


In [ ]:
# Load training and combined validation-test datasets from excel
# Note this was previously split to allow data augmentation of training data only

test_df = pd.read_csv(file_path + "TestData70-30.csv")

# Extract text and labels from datasets and split validation/test data
labels = ['Protocol', 'Urg', 'UrgBin']

y_train = combined_aug_train_df[labels]
y_test = test_df[labels]

print(combined_aug_train_df.shape)
print(y_train.shape)
print(test_df.shape)
print(y_test.shape)

In [ ]:
'''
EXPORT DATA
'''
combined_aug_train_df.to_csv(file_path + "Xtrain_aug.csv", encoding='utf-8', index=False)
test_df.to_csv(file_path + "Xtest_aug.csv", encoding='utf-8', index=False)
y_train.to_csv(file_path + "ytrain_aug.csv", encoding='utf-8', index=False)
y_test.to_csv(file_path + "ytest_aug.csv", encoding='utf-8', index=False)



In [ ]:
from google.colab import runtime
runtime.unassign()